# Imports

In [ ]:
import os
import numpy as np
import subprocess
import pandas as pd
import requests
from urllib.parse import urljoin
import sqlite3
import json
import time
from datetime import datetime, timedelta
import re
from google.colab import drive
import random
from bs4 import BeautifulSoup
import csv
!pip install pydriller
from pydriller import Repository
from urllib.parse import urlparse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00


In [ ]:
drive.mount('/content/gdrive/', force_remount=True)
# change location as per your convenience
# final_packages.txt (containing json dump of list of npm package names to be mined) should be present at this location
# database will be saved at this location
os.chdir("/content/gdrive/Shareddrives/ECS 260/final")

Mounted at /content/gdrive/


# Function Definitions

In [ ]:
def add_column_if_not_exists(cursor, table_name, column_name, column_definition):
    # check if the column already exists
    cursor.execute(f"PRAGMA table_info({table_name});")
    existing_columns = [column[1] for column in cursor.fetchall()]

    if column_name not in existing_columns:
        # add the column if it does not exist
        cursor.execute(f"ALTER TABLE {table_name} ADD COLUMN {column_name} {column_definition};")

In [ ]:
def GET(json_obj, keys, default = None):
  if not isinstance(keys, list):
    keys = [keys]
  current = json_obj
  try:
    for key in keys:
      if isinstance(current, list):
        key = int(key)
      current = current[key]
    return current
  except (TypeError, IndexError, KeyError):
    return default

def isValid(value):
  if isinstance(value, list):
    return not len(value) == 0
  return value != None and value != ""

def getN(arr):
  return len(arr) if isValid(arr) else None

In [ ]:
# fetch json from an API endpoint
def fetch_response_without_fail(url, params = {}, headers = {}, shouldScrap = False):
  retry_count = 0
  max_retries = 15
  time_interval = 10
  status = ""
  while retry_count < max_retries:
    try:
      response = requests.get(url, params=params, headers=headers)

      if response.status_code == 200:
        status = str(response.status_code)
        if shouldScrap:
          return (status, response)
        else:
          return (status, response.json())
      elif response.status_code == 404:
        status = str(response.status_code)
        print(f"Error: {response.status_code}, skipping url {url}")
        return (status, None)
      else:
        status = str(response.status_code)
        print(f"Error: {response.status_code}")
        print(params)
        retry_count += 1
        print(f"Retrying in {time_interval} seconds... (Retry {retry_count}/{max_retries})")
        if response.status_code == 429:
          retry_after = response.headers.get('Retry-After')
          if retry_after:
            print("Retrying after", retry_after + 5, "Seconds")
            time.sleep(int(retry_after) + 5)
          else:
            time.sleep(time_interval)
        else:
          time.sleep(time_interval)

    except Exception as e:
      status = f"Exception occurred: {e}"
      print(status)
      retry_count += 1
      print(f"Retrying in {time_interval} seconds... (Retry {retry_count}/{max_retries})")
      time.sleep(time_interval)

  print(f"Max retries reached. Unable to fetch data from {url}.")
  return (status, None)

In [ ]:
def get_lagging_dependencies_count(dependencies):
  status = "erred"
  if not isValid(dependencies):
    return (status, None)

  lagging_dependencies = 0
  for dependency, specified_version in dependencies.items():
    if specified_version == 'latest':
      continue

    (status, dependency_data) = fetch_response_without_fail(f'https://registry.npmjs.org/{dependency}/latest')
    if not status == '200':
      # skipping count if dependency doesn't exist for some reason
      continue
    latest_version = GET(dependency_data, 'version')
    if latest_version and specified_version != latest_version:
      lagging_dependencies += 1

  return ("success", lagging_dependencies)

In [ ]:
def extract_github_url(repo_url):
  if not isValid(repo_url):
    return None
  repo_url_lowered = repo_url.lower()
  if not repo_url_lowered.find('gist.github.com') == -1:
    return None
  if not repo_url_lowered.find('gitee.com') == -1:
    return None
  if not repo_url_lowered.find('bitbucket') == -1:
    return None

  github_idx = repo_url_lowered.find('github.com')
  if github_idx == -1:
    if repo_url.endswith('.git'):
      repo_url = repo_url[:-4]
      if ':' in repo_url:
        repo_url = repo_url.split(':')[-1]
      path_segments = repo_url.split('/')
      if len(path_segments) >= 2 and all(len(path) > 0 for path in path_segments[-2:]):
        author_repo = "/".join(path_segments[-2:])
        return f"https://github.com/{author_repo}"
      return None
    else:
      return None

  repo_url = repo_url[github_idx + len('github.com'):]
  path_segments = repo_url.split('/')
  if len(path_segments) >= 2 and all(len(path) > 0 for path in path_segments[-2:]):
    author_repo = "/".join(path_segments[-2:])
    author_repo = author_repo[:-4] if author_repo.endswith(".git") else author_repo
    return f"https://github.com/{author_repo}"
  else:
    return None

In [ ]:
extract_github_url('git+https://github.com/ibm/ibm-cos-sdk-js.git')

'https://github.com/ibm/ibm-cos-sdk-js'

In [ ]:
def extract_snyk_score(soup, category):
    selector = f'.scores li span:contains("{category}")'
    category_element = soup.select_one(selector)

    if category_element:
        category_score = category_element.find_next('span', class_='vue--pill__body').get_text(strip=True)
        return category_score.upper()
    else:
        return None

def scrap_snyk(package):
  status = "erred"
  try:
    status, response = fetch_response_without_fail(f"https://snyk.io/advisor/npm-package/{package}", shouldScrap=True)
    if not status == '200':
      return (status, None)
    response.raise_for_status() # Raise an HTTPError for bad responses
    soup = BeautifulSoup(response.text, 'html.parser')
    health_score_div = soup.select_one('.number span')
    if not health_score_div:
      if soup.find('div', class_='package-extra').find('p', class_='heading', string='This is a malicious package'):
        # For malicious packages, health score is 0
        health_score = 0
      else:
        # some error
        status = f"erred, element not found, status:{status}"
        health_score = None
      return (status, {
        "health_score": health_score,
        "security": None,
        "popularity": None,
        "maintenance": None,
        "community": None
      })
    health_score = health_score_div.get_text(strip=True).split('/')[0].strip()
    try:
      health_score = int(health_score)
    except:
      health_score

    return (status, {
      "health_score": health_score,
      "security": extract_snyk_score(soup, 'security'),
      "popularity": extract_snyk_score(soup, 'popularity'),
      "maintenance": extract_snyk_score(soup, 'maintenance'),
      "community": extract_snyk_score(soup, 'community')
    })
  except requests.exceptions.RequestException as e:
    status = str(response.status_code)
    print(f"Error scraping snyk for {package}: {e}")

  print(f"Error scraping snyk for {package}")
  return (status, None)

In [ ]:
def scrap_repo(repo_url):
  status = "erred"
  try:
    status, response = fetch_response_without_fail(repo_url, shouldScrap=True)
    if not status == '200':
      return (status, None)
    response.raise_for_status() # Raise an HTTPError for bad responses
    soup = BeautifulSoup(response.text, 'html.parser')
    border_grid_rows = soup.find_all(class_='BorderGrid-row')
    contributors = None
    for border_grid_row in border_grid_rows:
      text = border_grid_row.text.strip()
      if text.startswith('Contributors'):
        span_element = border_grid_row.find('span')
        if span_element:
          span_text = span_element.get_text(strip=True).replace(",", "")
          try:
            contributors = int(span_text)
          except:
            contributors = span_text

    commit_anchor_tag = soup.find("a", class_="react-last-commit-history-group")
    no_of_commits = None
    if commit_anchor_tag:
      # Find all span elements within the anchor tag
      spans = commit_anchor_tag.find_all("span")
      spans = spans[0].find_all("span")

      if len(spans) > 1:
        commit_count_span = spans[1]
        commit_count_text = commit_count_span.get_text(strip=True)
        no_of_commits = int(''.join(filter(str.isdigit, commit_count_text)))

    return (status, {"contributors": contributors, "no_of_commits": no_of_commits})
  except requests.exceptions.RequestException as e:
    status = str(response.status_code)
    print(f"Error scraping github repo {repo_url}: {e}")

  print(f"Error scraping github repo {repo_url}")
  return (status, None)

In [ ]:
def scrap_repo_net(repo_url):
  status = "erred"
  try:
    status, response = fetch_response_without_fail(repo_url + '/network/dependents', shouldScrap=True)
    if not status == '200':
      return (status, None)
    response.raise_for_status() # Raise an HTTPError for bad responses
    soup = BeautifulSoup(response.text, 'html.parser')
    btn_links = soup.find_all('a', class_='btn-link')
    dependants_count = None
    dependant_repos_count = None
    for btn_link in btn_links:
      text_content = btn_link.get_text(strip=True)

      if "Repositories" in text_content:
          dependants_count = int(''.join(filter(str.isdigit, (text_content.split()[0]))))
      elif "Packages" in text_content:
          dependant_repos_count = int(''.join(filter(str.isdigit, (text_content.split()[0]))))

    return (status, {"dependants_count": dependants_count, "dependant_repos_count": dependant_repos_count})
  except requests.exceptions.RequestException as e:
    status = str(response.status_code)
    print(f"Error scraping github repo {repo_url}: {e}")

  print(f"Error scraping github repo {repo_url}")
  return (status, None)

In [ ]:
def count_lines_of_code(repo_url):
  total_lines_of_code = 0
  filtered_lines_of_code = 0
  file_set = set()
  try:
    # Iterate over commits in the repository
    for commit in Repository(repo_url).traverse_commits():
      # Iterate over modified files in each commit
      for modified_file in commit.modified_files:
        # Check if the file is a source code file
        if modified_file.filename.endswith(('.py', '.json', '.md', '.html', '.yaml', '.yml', '.xml','.sh', '.css','.scss','.ts','.jsx','.tsx')):
          filtered_lines_of_code = filtered_lines_of_code+ (modified_file.added_lines - modified_file.deleted_lines)

        lines_of_code = modified_file.added_lines - modified_file.deleted_lines
        file_set.add(modified_file.filename)
        total_lines_of_code += lines_of_code
  except Exception as e:
    print("Didn't Work", repo_url)

  return total_lines_of_code, filtered_lines_of_code, len(file_set)

In [ ]:
def fetch_package_data_from_db(db_path, package_name, table_name):
  curr_conn = sqlite3.connect(db_path)
  curr_cursor = curr_conn.cursor()

  curr_cursor.execute(f"SELECT * FROM {table_name} WHERE package=?", (package_name,))
  curr_row = curr_cursor.fetchone()

  curr_conn.close()

  if curr_row:
    column_names = [description[0] for description in curr_cursor.description]
    package_data = dict(zip(column_names, curr_row))
    return package_data
  else:
    return None

# Distribution

In [ ]:
gh_token = '<your_github_access_token>'
li_api_key = '<your_librarie.io_api_token>'
# put index according to the no. of partitions
idx = 1

# final_packages.txt should have json dump of list of npm package names to be mined
with open('final_packages.txt', "r") as file:
  final_packages = file.read()
final_packages = json.loads(final_packages)

# specify how many parts you want to distribute the mining into
total_parts = 9
i = idx - 1
part_size = len(final_packages) // total_parts
package_list = final_packages[i * part_size:(i + 1) * part_size if i < total_parts - 1 else len(final_packages)]
print("Total rows to be mined: ", len(package_list))

Total rows to be mined:  3333


# DB Init

In [ ]:
# init database and cursor
conn = sqlite3.connect(f"database_{idx}.db")
cursor = conn.cursor()

In [ ]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS package_data (
        package TEXT
    )
""")

conn.commit()

In [ ]:
# columns other than `package`
columns = [
    ['npm_api_status', 'TEXT DEFAULT NULL'],
    # Def: response status from npm API
    #      possible values: 200 if everything is mined
    #                       404 if package is missing/not found
    #                       etc...
    # IMP: further mining is continued only if this status is 200

    ['latest_version', 'TEXT DEFAULT NULL'],
    # Def: latest version name of the package
    # Source: npm API

    ['no_of_versions', 'INT DEFAULT NULL'],
    # Def: total no of versions released for this package
    # Source: npm API, derived

    ['keywords', 'TEXT DEFAULT NULL'],
    # Def: json dump of list of keywords
    # Source: npm API

    ['no_of_users', 'INT DEFAULT NULL'],
    # Def: no of npm users who have starred this package w.r.t. npm (not github)
    # Source: npm API, derived

    ['has_readme', 'INT DEFAULT NULL'],
    # Def: if the package has a readme file provided with it
    # Source: npm API, derived

    ['has_homepage', 'INT DEFAULT NULL'],
    # Def: if the package has a homepage
    # Source: npm API, derived

    ['last_modified_timestamp', 'TEXT DEFAULT NULL'],
    # Def: timestamp when the package was last modified in npm registry
    # Source: npm API

    ['created_timestamp', 'TEXT DEFAULT NULL'],
    # Def: timestamp when the package was first published in npm registry
    # Source: npm API

    ['unpublished_timestamp', 'TEXT DEFAULT NULL'],
    # Def: timestamp when the package was unpublished from npm registry
    # Source: npm API
    # IMP: further mining is discontinued if this has been found valid
    #      as we do not get any repository url from this response

    ['version_history_timestamps', 'TEXT DEFAULT NULL'],
    # Def: json dump of dictionary of version-release_timestamp pairs
    # Source: npm API

    ['is_deprecated', 'INT DEFAULT NULL'],
    # Def: flag to denote if package is deprecated
    # Source: npm API, derived

    ['no_of_dependencies', 'INT DEFAULT 0'],
    # Def: total no. of dependencies and devDependencies
    # Source: npm API, derived

    ['lagging_dependencies_count', 'INT DEFAULT NULL'],
    # Def: count of lagging dependencies
    #      lagging dependency: dependency that is not the latest version of itself
    #                          for each dependency listed in the package's metadata,
    #                          compare the version specified in the package with
    #                          the latest available version.
    # Source: npm API, derived

    ['lagging_dependencies_count_status', 'INT DEFAULT NULL'],
    # Def: response status on calling lagging dependencies count function
    # Source: npm API, derived

    ['unpack_size', 'INT DEFAULT NULL'],
    # Def: the total byte of the unpacked files in the tarball
    # Source: npm API

    ['file_count', 'INT DEFAULT NULL'],
    # Def: the number of files in the tarball, folder excluded
    # Source: npm API

    ['has_repository', 'INT DEFAULT NULL'],
    # Def: if the package has a repository present (can be any type including github, bitbucket, etc.)
    # Source: npm API, derived

    ['repository_type', 'TEXT DEFAULT NULL'],
    # Def: type of repository as mentioned in metadata
    # Source: npm API

    ['repository_url', 'TEXT DEFAULT NULL'],
    # Def: url of repository as mentioned in metadata
    # Source: npm API

    ['git_repository_url', 'TEXT DEFAULT NULL'],
    # Def: github repository url, formatted properly
    # Source: npm API, derived using extract_github_url(repository_url)

    #####################################################################################################################

    ['li_api_status', 'TEXT DEFAULT NULL'],
    # Def: response status from calling libraries.io API
    #      possible values: 200 if API working fine
    #                       404 if API resource not found
    #                       etc...
    # Source: libraries.io API

    ['rank', 'TEXT DEFAULT NULL'],
    # Def: popularity rank
    # Source: libraries.io API

    ['dependants_count', 'TEXT DEFAULT NULL'],
    # Def: no. of packages that are dependent on current package
    #      this is overwritten if its obtained from github page scrapping later on
    # Source: libraries.io API

    ['dependant_repos_count', 'TEXT DEFAULT NULL'],
    # Def: no. of repositories that are dependent on current package
    #      this is overwritten if its obtained from github page scrapping later on
    # Source: libraries.io API

    #####################################################################################################################

    ['git_repository_url_final', 'TEXT DEFAULT NULL'],
    # Def: final github repository url after accounting for any redirects in git_repository_url
    # Source: git_repository_url, derived

    ['git_repository_status', 'TEXT DEFAULT NULL'],
    # Def: response status from calling git_repository_url_final
    #      possible values: 200 if repo working fine
    #                       404 if repo not found, possibly made private or removed
    #                       etc...
    # IMP: further github mining is continued only if this status is 200

    #####################################################################################################################

    ['gh_api_status', 'TEXT DEFAULT NULL'],
    # Def: response status from calling github API
    #      possible values: 200 if API working fine
    #                       404 if API resource not found
    #                       etc...
    # IMP: forks, stars, watchers, avg_commit_freq is mined correctly if this status is 200
    # Source: github API

    ['forks', 'INT DEFAULT NULL'],
    # Def: no. of forks
    # Source: github API

    ['stars', 'INT DEFAULT NULL'],
    # Def: no. of stars
    # Source: github API

    ['watchers', 'INT DEFAULT NULL'],
    # Def: no. of watchers
    # Source: github API

    #####################################################################################################################

    ['gh_issue_api_status', 'TEXT DEFAULT NULL'],
    # Def: response status from calling github issues API
    #      possible values: 200 if API working fine
    #                       404 if API resource not found
    #                       etc...
    # IMP: issues is mined correctly if this status is 200
    # Source: github API

    ['issues', 'INT DEFAULT NULL'],
    # Def: latest version name of the package
    # Source: github issues API

    #####################################################################################################################

    ['gh_pr_api_status', 'TEXT DEFAULT NULL'],
    # Def: response status from calling github pr API
    #      possible values: 200 if API working fine
    #                       404 if API resource not found
    #                       etc...
    # IMP: pr is mined correctly if this status is 200
    # Source: github pr API

    ['pr', 'INT DEFAULT NULL'],
    # Def: latest version name of the package
    # Source: github API

    #####################################################################################################################

    ['gh_scrapping_status', 'TEXT DEFAULT NULL'],
    # Def: response status from calling github repository page of current package
    #      possible values: 200 if webpage working fine
    #                       404 if webpage not found
    #                       etc...
    # Source: github repository page scrapping

    ['contributors', 'INT DEFAULT NULL'],
    # Def: no. of contributors in current package's repository
    # Source: github repository page scrapping

    ['no_of_commits', 'INT DEFAULT NULL'],
    # Def: total no. of commits in current package's repository
    # Source: github repository page scrapping

    ['avg_commit_freq', 'REAL DEFAULT NULL'],
    # Def: frequency of commit w.r.t lifespan of repository, in no. per days
    # Source: github repository page scrapping and created_at from github API, derived

    #####################################################################################################################

    ['gh_net_scrapping_status', 'TEXT DEFAULT NULL'],
    # Def: response status from calling github repository's network page scrapping
    #      possible values: 200 if webpage working fine
    #                       404 if webpage not found
    #                       etc...
    # Source: github repository's network page scrapping

    ['dependants_count', 'INT DEFAULT NULL'],
    # Def: no. of packages that are dependent on current package
    # Source: github repository's network page scrapping

    ['dependant_repos_count', 'INT DEFAULT NULL'],
    # Def: no. of repositories that are dependent on current package
    # Source: github repository's network page scrapping

    #####################################################################################################################

    ['total_lines_of_code', 'INT DEFAULT NULL'],
    # Def: LOC count
    # Source: pydriller

    ['filtered_lines_of_code', 'INT DEFAULT NULL'],
    # Def: filtered LOC count
    # Source: pydriller

    ['no_of_files', 'INT DEFAULT NULL'],
    # Def: no. of files
    # Source: pydriller

    #####################################################################################################################

    ['snyk_scrapping_status', 'TEXT DEFAULT NULL'],
    # Def: response status from calling snyk package advisor webpage
    #      possible values: 200 if webpage working fine
    #                       404 if webpage not found
    #                       etc...
    # Source: snyk package advisor webpage scrapping

    ['health_score', 'INT DEFAULT NULL'],
    # Def: comprehensive evaluation of an npm package's overall health, considering multiple factors such as quality,
    #      security, and maintenance. It aids developers in assessing the package's overall quality and reliability
    # Source: snyk package advisor webpage scrapping

    ['security', 'TEXT DEFAULT NULL'],
    # Def: evaluation of package vulnerability, considering the presence and severity of known security issues
    # Source: snyk package advisor webpage scrapping

    ['popularity', 'TEXT DEFAULT NULL'],
    # Def: metric that represents the widespread use of a package in the development community, indicating stability and maintenance
    # Source: snyk package advisor webpage scrapping

    ['maintenance', 'TEXT DEFAULT NULL'],
    # Def: metric to measure the responsiveness of package maintainers to issues, updates, and community interactions, enhancing stability
    # Source: snyk package advisor webpage scrapping

    ['community', 'TEXT DEFAULT NULL'],
    # Def: metric to represent the level of community support and engagement, contributing to the overall health of the package
    # Source: snyk package advisor webpage scrapping
]

# adding columns to the table
for col in columns:
  add_column_if_not_exists(cursor, 'package_data', col[0], col[1])

conn.commit()

In [ ]:
df = pd.read_sql_query(f"SELECT * FROM package_data", conn)
df

,package,npm_api_status,latest_version,no_of_versions,keywords,no_of_users,has_readme,has_homepage,last_modified_timestamp,created_timestamp,...,gh_net_scrapping_status,total_lines_of_code,filtered_lines_of_code,no_of_files,snyk_scrapping_status,health_score,security,popularity,maintenance,community
0,le-shared-utils-teste-app,200,0.0.0,1.0,None,NaN,1.0,0.0,2022-05-07T18:19:55.077Z,2019-08-08T12:41:10.380Z,...,None,NaN,NaN,NaN,200,28,SECURITY REVIEW NEEDED,LIMITED,INACTIVE,LIMITED
1,data-fetch-manager-entry-service,200,1.1.0,3.0,None,NaN,1.0,1.0,2023-12-10T15:39:11.400Z,2023-12-10T13:48:40.949Z,...,200,463.0,312.0,15.0,404,None,None,None,None,None
2,@hyper.fun/material-icon-dns-outlined,200,1.3.0,2.0,"[""hyper-function-component"", ""hfc"", ""icon"", ""m...",NaN,0.0,0.0,2022-08-06T00:34:42.126Z,2022-07-28T04:31:03.742Z,...,None,NaN,NaN,NaN,200,?,PENDING…,PENDING…,PENDING…,PENDING…
3,xls2enketo,200,0.0.1,3.0,None,NaN,1.0,0.0,2022-05-25T00:00:44.901Z,2019-08-30T09:16:27.067Z,...,None,NaN,NaN,NaN,200,34,SECURITY REVIEW NEEDED,LIMITED,INACTIVE,LIMITED
4,stylie.treeview,200,1.0.1,2.0,"[""navigation"", ""file browser"", ""menu"", ""css"", ...",NaN,1.0,1.0,2022-06-27T01:41:31.505Z,2015-05-04T22:23:39.685Z,...,200,6706.0,1709.0,71.0,200,42,NO KNOWN SECURITY ISSUES,LIMITED,INACTIVE,LIMITED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,openapiproto,200,1.0.0,1.0,None,NaN,0.0,0.0,2022-05-12T06:07:38.662Z,2020-01-23T22:44:21.347Z,...,None,NaN,NaN,NaN,200,40,NO KNOWN SECURITY ISSUES,LIMITED,INACTIVE,LIMITED
3329,taoladat,200,1.0.0,1.0,None,NaN,1.0,1.0,2022-05-19T09:32:13.837Z,2021-01-06T07:01:11.816Z,...,None,NaN,NaN,NaN,200,40,NO KNOWN SECURITY ISSUES,LIMITED,INACTIVE,LIMITED
3330,pouchdb-utils-sapper,200,0.0.3,3.0,None,NaN,1.0,0.0,2022-05-13T07:29:40.911Z,2020-04-23T15:19:21.143Z,...,None,NaN,NaN,NaN,200,40,NO KNOWN SECURITY ISSUES,LIMITED,INACTIVE,LIMITED
3331,@react-goosebumps/react-cli,200,0.1.0,1.0,None,NaN,1.0,0.0,2022-04-06T18:23:50.160Z,2018-07-29T20:48:23.968Z,...,None,NaN,NaN,NaN,200,?,PENDING…,PENDING…,PENDING…,PENDING…


# Mining

In [ ]:
cursor.execute("SELECT package FROM package_data")
rows = cursor.fetchall()
rows_fetched = len(rows)
print(f"Rows mined so far: {len(rows)}/{len(package_list)}")

Rows mined so far: 3333/3333


In [ ]:
cursor.execute("SELECT package FROM package_data")
rows = cursor.fetchall()
rows_fetched = len(rows)

for package in package_list[rows_fetched:]:
  print(f"Now mining\t{package}")
  final_data = {'package': package}

  # mining npm API data
  npm_api_url = f"https://registry.npmjs.org/{package}"
  npm_api_res = fetch_response_without_fail(npm_api_url, {})

  npm_api_status, npm_api_res = fetch_response_without_fail(npm_api_url)
  final_data['npm_api_status'] = npm_api_status

  continue_mining = npm_api_status == "200"

  if continue_mining:
    last_modified_timestamp = GET(npm_api_res, ["time", "modified"])
    final_data['last_modified_timestamp'] = last_modified_timestamp

    created_timestamp = GET(npm_api_res, ["time", "created"])
    final_data['created_timestamp'] = created_timestamp

    unpublished_timestamp = GET(npm_api_res, ["time", "unpublished", "time"])
    final_data['unpublished_timestamp'] = unpublished_timestamp

    version_history_timestamps = json.dumps({key: value for key, value in GET(npm_api_res, "time").items() if key not in ["modified", "created", "unpublished"]})
    final_data['version_history_timestamps'] = version_history_timestamps

    no_of_versions = len(json.loads(version_history_timestamps)) if isValid(version_history_timestamps) else None
    final_data['no_of_versions'] = no_of_versions

    if not isValid(unpublished_timestamp):
      latest_version = GET(npm_api_res, ["dist-tags", "latest"])
      final_data['latest_version'] = latest_version

      versions = GET(npm_api_res, "versions")

      keywords = GET(versions, [latest_version, 'keywords'])
      final_data['keywords'] = json.dumps(keywords) if isValid(keywords) else None

      no_of_users = getN(GET(npm_api_res, 'users'))
      final_data['no_of_users'] = no_of_users

      has_readme = isValid(GET(npm_api_res, "readmeFilename"))
      final_data['has_readme'] = has_readme

      has_homepage = isValid(GET(npm_api_res, "homepage"))
      final_data['has_homepage'] = has_homepage

      is_deprecated = isValid(GET(versions, [latest_version, 'deprecated']))
      final_data['is_deprecated'] = is_deprecated

      unpack_size = GET(versions, [latest_version, 'dist', 'unpackedSize'])
      final_data['unpack_size'] = unpack_size

      file_count = GET(versions, [latest_version, 'dist', 'fileCount'])
      final_data['file_count'] = file_count

      dependencies = GET(versions, [latest_version, 'dependencies'], {})
      dev_dependencies = GET(versions, [latest_version, 'devDependencies'], {})
      no_of_dependencies = len(dependencies) + len(dev_dependencies)
      final_data['no_of_dependencies'] = no_of_dependencies

      stat, lagging_dependencies_count = get_lagging_dependencies_count(dependencies)
      stat_dev, lagging_dev_dependencies_count = get_lagging_dependencies_count(dev_dependencies)
      if stat == 'success' and stat_dev == 'success':
        final_data['lagging_dependencies_count_status'] = 'success'
        final_data['lagging_dependencies_count'] = lagging_dependencies_count + lagging_dev_dependencies_count
      else:
        final_data['lagging_dependencies_count_status'] = 'erred'

      has_repository = isValid(GET(npm_api_res, ["repository"]))
      final_data['has_repository'] = has_repository

      repository_type = GET(npm_api_res, ["repository", "type"])
      final_data['repository_type'] = repository_type

      repository_url = GET(npm_api_res, ["repository", "url"])
      final_data['repository_url'] = repository_url

      git_repository_url = extract_github_url(repository_url)
      final_data['git_repository_url'] = git_repository_url

  print("NPM API mined")
  continue_mining = continue_mining and not isValid(unpublished_timestamp)
  if continue_mining:
    # mining libraries.io API data
    li_api_url = f"https://libraries.io/api/npm/{package}"
    li_api_status, li_api_res = fetch_response_without_fail(li_api_url, {'api_key': li_api_key})
    final_data['li_api_status'] = li_api_status

    if li_api_status == '200':
      rank = GET(li_api_res, "rank")
      final_data['rank'] = rank

      dependants_count = GET(li_api_res, "dependents_count")
      final_data['dependants_count'] = dependants_count

      dependant_repos_count = GET(li_api_res, "dependent_repos_count")
      final_data['dependant_repos_count'] = dependant_repos_count

    print("Libraries.io API mined")
    # scrapping snyk.io data
    snyk_scrapping_status, snyk_scrapping_res = scrap_snyk(package)
    final_data['snyk_scrapping_status'] = snyk_scrapping_status

    if snyk_scrapping_status == '200':
      final_data['health_score'] = snyk_scrapping_res['health_score']
      final_data['security'] = snyk_scrapping_res['security']
      final_data['popularity'] = snyk_scrapping_res['popularity']
      final_data['maintenance'] = snyk_scrapping_res['maintenance']
      final_data['community'] = snyk_scrapping_res['community']

    print("snyk.io API mined")
    # mining github API data
    # getting redirected gh url
    if isValid(git_repository_url):
      try:
        git_repository_url_final = requests.get(git_repository_url, allow_redirects=False).headers['Location']
      except Exception as e:
        git_repository_url_final = git_repository_url

      # checking if url is valid
      try:
        res = requests.get(git_repository_url_final)
        git_repository_status = str(res.status_code)
      except Exception as e:
        git_repository_status = f'Exception: {e}'
      final_data['git_repository_status'] = git_repository_status
      continue_mining = continue_mining and git_repository_status == '200'
    else:
      continue_mining = False

  # continue github mining
  if continue_mining:
    print("Git URL OK,  Continuing...")
    final_data['git_repository_url_final'] = git_repository_url_final

    headers = {
      'Authorization': f'token {gh_token}',
      'Accept': 'application/vnd.github.v3+json'
    }

    repo_owner, repo_name = git_repository_url_final.split('/')[-2:]
    gh_api_url = f"https://api.github.com/repos/{repo_owner}/{repo_name}"

    # mining github API data
    gh_api_status, gh_api_res = fetch_response_without_fail(gh_api_url, headers=headers)
    final_data['gh_api_status'] = gh_api_status
    if gh_api_status == '200':
      forks = GET(gh_api_res, 'forks_count')
      final_data['forks'] = forks

      stars = GET(gh_api_res, 'stargazers_count')
      final_data['stars'] = stars

      watchers = GET(gh_api_res, 'subscribers_count')
      final_data['watchers'] = watchers

      start_date_timestamp = GET(gh_api_res, 'created_at')
      if isValid(start_date_timestamp):
        start_date = datetime.strptime(start_date_timestamp, "%Y-%m-%dT%H:%M:%SZ").date()
        # age till Feb 23, 2024
        end_date = datetime(2024, 2, 23).date()
        total_days = (end_date - start_date).days
      else:
        total_days = None


    # mining github issues API data
    gh_issues_api_url = f"https://api.github.com/search/issues?q=is:issue%20is:open%20repo:{repo_owner}/{repo_name}"
    gh_issue_api_status, gh_issues_api_res = fetch_response_without_fail(gh_issues_api_url, headers={'Authorization': f'token {gh_token}'})
    final_data['gh_issue_api_status'] = gh_issue_api_status
    if gh_issue_api_status == '200':
      issues = GET(gh_issues_api_res, 'total_count')
      final_data['issues'] = issues


    # mining github pr API data
    gh_pr_api_url = f"https://api.github.com/search/issues?q=is:pr%20is:open%20repo:{repo_owner}/{repo_name}"
    gh_pr_api_status, gh_pr_api_res = fetch_response_without_fail(gh_pr_api_url, headers={'Authorization': f'token {gh_token}'})
    final_data['gh_pr_api_status'] = gh_pr_api_status
    if gh_pr_api_status == '200':
      pr = GET(gh_pr_api_res, 'total_count')
      final_data['pr'] = pr


    # scrapping repository data
    gh_scrapping_status, gh_scrapping_res = scrap_repo(git_repository_url_final)
    final_data['gh_scrapping_status'] = gh_scrapping_status

    if gh_scrapping_status == '200':
      final_data['contributors'] = gh_scrapping_res['contributors']
      final_data['no_of_commits'] = gh_scrapping_res['no_of_commits']
      final_data['avg_commit_freq'] = gh_scrapping_res['no_of_commits'] / total_days if isValid(total_days) else None


    # scrapping repository network data
    gh_net_scrapping_status, gh_net_scrapping_res = scrap_repo_net(git_repository_url_final)
    final_data['gh_net_scrapping_status'] = gh_net_scrapping_status

    if gh_net_scrapping_status == '200':
      final_data['dependants_count'] = gh_net_scrapping_res['dependants_count']
      final_data['dependant_repos_count'] = gh_net_scrapping_res['dependant_repos_count']

    # pydriller
    try:
      old_data = fetch_package_data_from_db('FINAL_PACKAGE_DATA.db', package, 'final_package_data')
      if isValid(old_data) and (isValid(old_data['Lines Of Codes']) or isValid(old_data['Number of Files']) or isValid(old_data['Filtered Lines of Codes'])):
        final_data['total_lines_of_code'] = old_data['Lines Of Codes']
        final_data['filtered_lines_of_code'] = old_data['Number of Files']
        final_data['no_of_files'] = old_data['Filtered Lines of Codes']
      else:
        # some repo urls have been same for multiple packages so fetching data from them instead of recalculating
        cursor.execute("SELECT total_lines_of_code, filtered_lines_of_code, no_of_files FROM package_data WHERE git_repository_url_final = ? AND (total_lines_of_code IS NOT NULL OR filtered_lines_of_code IS NOT NULL OR no_of_files IS NOT NULL)", [git_repository_url_final])
        result_to_check = cursor.fetchone()
        if isValid(result_to_check):
          final_data['total_lines_of_code'], final_data['filtered_lines_of_code'], final_data['no_of_files'] = result_to_check
        else:
          # If the result is empty or values are None, calculate the lines of code
          total_lines_of_code, filtered_lines_of_code, no_of_files = count_lines_of_code(git_repository_url_final)
          final_data['total_lines_of_code'] = total_lines_of_code
          final_data['filtered_lines_of_code'] = filtered_lines_of_code
          final_data['no_of_files'] = no_of_files

    except Exception as e:
      print(f"pydriller failed for {package}\n{e}")
  else:
    print("Git URL Not OK,  Skipping...")

  if len(final_data) > 0:
    columns = ', '.join(final_data.keys())
    values = ', '.join(['?' for _ in final_data.values()])

    # Execute the INSERT statement
    cursor.execute(f"INSERT INTO package_data ({columns}) VALUES ({values})", tuple(final_data.values()))

    # Commit the changes and close the database connection
    conn.commit()

  rows_fetched += 1
  print("--------------------------------------------------------------------------------------------------------------------------------------------")
  print(f"Rows fetched so far: {rows_fetched}/{len(package_list)}")
  print("--------------------------------------------------------------------------------------------------------------------------------------------")
  print()

In [ ]:
conn.close()